### Data et modélisation

**Axes d'améliorations techniques :**

- Source de données : Menus créés par des diététiciens et nutritionistes
- Adapter les formules de calcul des valeurs nutritionelles quotidienne et leurs paramètres : Basé sur un état de l'art de la littérature scientifique sur le sujet et les recommandations de diététiciens et nutritionistes
- Algorithme de création des menus : Utiliser une méthode plus robuste, basée sur plus d'indicateurs nutritionels et plus de recettes. L'utilisation d'une IA est envisageable.

**Axes d'améliorations fonctionnelles :**

- Saisonalité : Privilégier les plats de saison
- Options de dietes et restrictions plus variées : Prendre en compte des régimes religieux ou plus spécifiques, permettre à l'utilisateur de blacklister des ingrédients (même non allergènes) afin de filtrer toutes les recettes en contenant 
- Adapter les quantité des recettes en fonction du nombre de parts demandé

### Algorithme de génération des menus

**Input :** 

- recipes : json contenant les recettes, nécessaire (pas de valeur par défaut)
- diet : régime de l'utilisateur ('omnivore', 'pescitarian', 'vegetarian', 'vegan'), 'omnivore' par défaut
- tags : liste des restrictions alimentaires de l'utilisateur ('contain_gluten', 'contain_dairy', 'contain_nuts', 'contain_eggs', 'contain_soy', 'high_FODMAP'), [] par défaut
- weight : masse de l'utilisateur en kg, nécessaire (pas de valeur par défaut)
- height : taille de l'utilisateur en cm, nécessaire (pas de valeur par défaut)
- age : age de l'utilisateur en années, nécessaire (pas de valeur par défaut)
- goal : objectif de l'utilisateur ('maintain', 'weight loss', 'weight gain'), 'maintain' par défaut
- servings : nombre de parts voulu (entier), 1 par défaut (non utilisé)
- gender : genre ('male', 'female'), 'male' par défaut
- activity_level : niveau d'activité physique de l'utilisateur ('sedentary', 'lightly active', 'moderately active', 'very active', 'super active'), 'moderatly active' par défaut

**Ouput :**

- Une version filtrée de recipes contenant 1 petit-déjeuné, 2 entrée, 2 plats et 2 dessert, correspondant aux repas d'une journée

#### Fonctionnement

- A partir des données personnelles de l'utilisateur et de son but, des valeurs nutritionelles quotidiennes recommandées sont calculées
- Un ensemble de plats formant trois repas pour une journée est séléectionné aléatoirement
- Un score évaluant le respect du menu des valeurs nutritionelles est calculé
- Si ce score atteint un certain seuil, le menu est renvoyé à l'utilisateur
- Sinon, d'autres menus générés aléatoirements sont testés pendant 20 secondes jusqu'à l'obtention d'un menu dont le score atteint le seuil
- Si le score d'aucun menu testé atteint le seuil, le menu dont le score est le plus élévé est renvoyé à l'utilisateur 

For men:

- BMR = 10 × weight (kg) + 6.25 × height (cm) − 5 × age (years) + 5

For women:

- BMR = 10 × weight (kg) + 6.25 × height (cm) − 5 × age (years) − 161

- Sedentary (little or no exercise): BMR x 1.2
- Lightly active (light exercise/sports 1-3 days/week): BMR x 1.375
- Moderately active (moderate exercise/sports 3-5 days/week): BMR x 1.55
- Very active (hard exercise/sports 6-7 days a week): BMR x 1.725
- Super active (very hard exercise/physical job): BMR x 1.9

In [181]:
import numpy as np
import pandas as pd
import json

def show_df(df):
    print(df.shape)
    display(pd.concat([df.head(2), df.tail(1)]))

In [182]:
def calculate_bmr(weight, height, age, gender):
    if gender == 'male':
        return 10 * weight + 6.25 * height - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height - 5 * age - 161

def calculate_tdee(bmr, activity_level):
    activity_factors = {
        'sedentary': 1.2,
        'lightly active': 1.375,
        'moderately active': 1.55,
        'very active': 1.725,
        'super active': 1.9
    }
    return bmr * activity_factors[activity_level]

def adjust_for_goal(tdee, goal):
    if goal == 'maintain':
        return tdee
    elif goal == 'weight loss':
        return tdee - 500
    elif goal == 'weight gain':
        return tdee + 500

def daily_nutritional_requirements(weight, height, age, gender, activity_level, goal):
    bmr = calculate_bmr(weight, height, age, gender)
    tdee = calculate_tdee(bmr, activity_level)
    adjusted_calories = adjust_for_goal(tdee, goal)

    # Macronutrient distribution
    carbs = adjusted_calories * 0.55 / 4  # 55% of calories from carbs, 4 kcal/g
    protein = adjusted_calories * 0.20 / 4  # 20% of calories from protein, 4 kcal/g
    fats = adjusted_calories * 0.25 / 9  # 25% of calories from fats, 9 kcal/g

    return {
        'calories': round(adjusted_calories),
        'carbohydrates': round(carbs),
        'protein': round(protein),
        'fats': round(fats)
    }

# Example usage
requirements = daily_nutritional_requirements(70, 175, 25, 'male', 'moderately active', 'maintain')
print(requirements)


{'calories': 2594, 'carbohydrates': 357, 'protein': 130, 'fats': 72}


In [189]:
df = pd.read_json("/home/camille/code/Camille9999/EPSI/workshop-m1/data/data/generated/recipes_translated.json").T.reset_index(names='uuid')
show_df(df)

(1045, 13)


,uuid,recipe_name,total_time_minutes,servings,ingredients,directions,rating,url,recipe_type,nutrition,img_src,diet,tags
0,c4dfd4b8-af04-4a53-9cfd-51b83123555b,Tarte aux pommes de grand-mère Ople,90,8,"{'petites pommes': {'amount': 8.0, 'unit': Non...","Peler et épépiner les pommes, puis les tranche...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,Desserts,"{'Total Fat': {'amount': '19g', 'percent': '24...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...,vegetarian,"[contains_dairy, contains_gluten, high_FODMAP]"
1,4816ee40-3c66-4536-a551-e042b9ab362d,Compote de pommes maison de Sarah,25,4,"{'pommes': {'amount': 4.0, 'unit': None}, 'eau...","Mélangez les pommes, l'eau, le sucre et la can...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,Plat d'accompagnement,"{'Total Fat': {'amount': '0g', 'percent': '0%'...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...,vegan,[high_FODMAP]
1044,52551a0e-cb08-4df0-bc47-25e1e8d3794f,Barres énergétiques au chocolat du chef John,180,12,"{'dattes': {'amount': 480.0, 'unit': 'ml'}, 'n...","Mettre les noix de cajou, les amandes, la noix...",4.8,https://www.allrecipes.com/recipe/254452/chef-...,Apéritifs et collations,"{'Total Fat': {'amount': '22g', 'percent': '28...",https://www.allrecipes.com/thmb/IOoj42MMo8YNKW...,vegan,[contains_nuts]


In [185]:
with open("/home/camille/code/Camille9999/EPSI/workshop-m1/data/data/generated/recipes_translated.json", 'r') as f:
    recipes = json.load(f)

In [186]:
def filter_recipes(recipes, diet, tags):
    # Define dietary categories
    dietary_categories = {
        'omnivore': ['omnivore', 'pescatarian', 'vegetarian', 'vegan'],
        'pescatarian': ['pescatarian', 'vegetarian', 'vegan'],
        'vegetarian': ['vegetarian', 'vegan'],
        'vegan': ['vegan']
    }

    # Filter recipes based on diet
    filtered_recipes = {uuid : recipe for uuid, recipe in recipes.items() if recipe['diet'] in dietary_categories[diet]}

    # Filter recipes based on allergens
    filtered_recipes = {uuid : recipe for uuid, recipe in filtered_recipes.items() if not any(tag in recipe['tags'] for tag in tags)}

    return filtered_recipes


In [187]:
user_diet = 'vegetarian'
user_allergens = ['contain_soy', 'contain_gluten']

filtered_recipes = filter_recipes(recipes, user_diet, user_allergens)
len(recipes), len(filtered_recipes)

(1045, 867)

In [171]:
import random
import time

def parse_nutritional_values(nutrition, recipe_servings, servings):
    parsed_values = {}
    for key, value in nutrition.items():
        amount = value['amount']
        if 'mg' in amount:
            parsed_values[key] = servings * (float(amount.replace('mg', '')) / 1000) / recipe_servings
        elif 'g' in amount:
            parsed_values[key] = servings * (float(amount.replace('g', ''))) / recipe_servings
        elif 'kcal' in amount:
            parsed_values[key] = servings * (float(amount.replace('kcal', ''))) / recipe_servings
    return parsed_values

def find_meals(recipes, daily_requirements, servings, margin=0.1):

    daily_requirements = {f"Total {key.capitalize()[:-1]}" if key in ['fats', 'carbohydrates'] else key.capitalize(): value * servings for key, value in daily_requirements.items()}

    categories = {
        "breakfast": ["Petit déjeuner et brunch"],
        "appetizer": ["Salade", "Apéritifs et collations", "Recettes de soupes, ragoûts et chili", "Recettes de soupes"],
        "main_course": ["Plat d'accompagnement", "Plats principaux", "Viandes et volailles", "Fruit de mer", "Barbecue et grillades"],
        "desert": ["Desserts"]
    }

    selected_meals = {
        "breakfast": None,
        "appetizer_1": None,
        "main_course_1": None,
        "desert_1": None,
        "appetizer_2": None,
        "main_course_2": None,
        "desert_2": None
    }

    # Filter recipes by category
    categorized_recipes = {category: {} for category in categories}
    for recipe_id, recipe in recipes.items():
        for category, types in categories.items():
            if recipe['recipe_type'] in types:
                categorized_recipes[category][recipe_id] = recipe

    # Helper function to check if nutritional values are within the margin
    def within_margin(total_nutritional_values, daily_requirements):
        keys_to_check = ['Calories', 'Total Carbohydrate', 'Protein', 'Total Fat']
        abs_differences = []
        for key in keys_to_check:
            if key in total_nutritional_values:
                abs_differences.append(np.abs((total_nutritional_values[key] - daily_requirements[key]) / daily_requirements[key]))
            else:
                abs_differences.append(0)
        score = np.average(abs_differences, weights=[4, 1, 1, 1])
        return score


    # Initialize total nutritional values
    total_nutritional_values = {
        'Calories': 0,
        'Total Carbohydrate': 0,
        'Protein': 0,
        'Total Fat': 0
    }

    def update_nutri(nutritional_values):
        for key in total_nutritional_values.keys():
            total_nutritional_values[key] += nutritional_values.get(key, 0)

    def select_meal(category, num_meals=1):
        meals = [(recipe_id, recipe) for recipe_id, recipe in categorized_recipes[category].items()]
        random.shuffle(meals)
        for i in range(num_meals):
            recipe_id, recipe = meals[i]
            nutritional_values = parse_nutritional_values(recipe['nutrition'], recipe['servings'], servings)
            selected_meals[f'{category}_{i+1}' if num_meals > 1 else category] = recipe_id
            update_nutri(nutritional_values)

    select_meal('breakfast', 'breakfast')
    select_meal('appetizer', num_meals=2)
    select_meal('main_course', num_meals=2)
    select_meal('desert', num_meals=2)

    score = within_margin(total_nutritional_values, daily_requirements)
    return selected_meals, score, total_nutritional_values


def main():

    max_duration = 20
    start_time = time.time()

    daily_requirements = {
        "calories": 2100,
        "carbohydrates": 356,
        "protein": 129,
        "fats": 72
    }
    selected_meals = None
    margin = 0.2
    best_score = 1000
    best_meal = None
    best_nutri = None
    while selected_meals == None:
        selected_meals, score, total_nutri = find_meals(recipes, daily_requirements, 1)
        if score < margin:
            return selected_meals, total_nutri
        elif score < best_score:
            best_score = score
            best_meal = selected_meals
            best_nutri = total_nutri
        selected_meals = None
        elapsed_time = time.time() - start_time
        if elapsed_time > max_duration:
            print("Best score:", best_score)
            return best_meal, best_nutri


In [172]:
meal, nutri = main()
nutri

{'Calories': 2105.125,
 'Total Carbohydrate': 227.95833333333334,
 'Protein': 80.16666666666667,
 'Total Fat': 74.625}

In [173]:
meal

{'breakfast': '0a42ed5f-231d-4cba-b3cf-14e2eab870e9',
 'appetizer_1': '04304f83-a7f2-41d5-a0dc-76d1cc421eb0',
 'main_course_1': 'e7d10045-2d5e-4194-a1b4-cd9c7399aaa7',
 'desert_1': '34e5e8b6-0e5b-4075-8a01-4354beb5d3f4',
 'appetizer_2': '7fdcfbf4-7bfe-4de7-bf00-68c7b0a97255',
 'main_course_2': '99ef7e53-241d-47b2-b64a-62d6ce961056',
 'desert_2': 'c8effa0f-ee24-4e3d-a070-61f67a3f5ebe'}

In [180]:
recipes[meal.get('desert_2')]

{'recipe_name': 'Tarte aux pêches avec pêches surgelées',
 'total_time_minutes': 130,
 'servings': 8,
 'ingredients': {'pâte à tarte en feuilles': {'amount': 2.0, 'unit': None},
  'farine tout usage': {'amount': 10.0, 'unit': 'ml'},
  'pêches surgelées': {'amount': 907.0, 'unit': 'g'},
  'sucre blanc': {'amount': 15.0, 'unit': 'ml'},
  'jus de citron': {'amount': 5.0, 'unit': 'ml'},
  'cannelle moulue': {'amount': 2.0, 'unit': 'ml'},
  'sel': {'amount': 1.0, 'unit': 'ml'},
  'fécule de maïs': {'amount': 44.0, 'unit': 'ml'},
  'gros œuf': {'amount': 1.0, 'unit': None},
  'sucre pétillant grossier': {'amount': 15.0, 'unit': 'ml'}},
 'directions': "Laissez reposer les croûtes à tarte à température ambiante pendant environ 15 minutes si le fabricant le recommande.\nPréchauffez le four à 400 degrés F (200 degrés C).\nRetirez l'emballage en plastique des croûtes à tarte et déroulez délicatement une pièce sur un plan de travail fariné. Saupoudrez la pâte de farine des deux côtés. Placez la pâ